# Self-Driving Car Engineering
## Vehicle Detection and Tracking
A pipeline to detect and track cars in a video stream from an autonomous car's forward facing camera

## Step 1: Build and train a classifier to detect cars in an image

In [1]:
import glob
import matplotlib.image as mpimg
import numpy as np
from skimage.feature import hog

# Extract the image names
cars = glob.glob("./vehicles/*.png")
notcars = glob.glob("./non-vehicles/*.png")

# define a few parameters for HOG feature extraction
colorspace = 'RGB' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9
pix_per_cell = 8
cell_per_block = 2
hog_channel = 0 # Can be 0, 1, 2, or "ALL"

#### Define a few functions to extract HOG features from a list of images

In [2]:
# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), block_norm= 'L2-Hys',
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), block_norm= 'L2-Hys',
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, cspace='RGB', orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif cspace == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      

        # Call get_hog_features() with vis=False, feature_vec=True
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))
            hog_features = np.ravel(hog_features)        
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        # Append the new feature vector to the features list
        features.append(hog_features)
    # Return list of feature vectors
    return features


#### Extract all car and non-car features into their respective lists

In [3]:
import time

# extract HOG features
t=time.time()
car_features = extract_features(cars, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
notcar_features = extract_features(notcars, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to extract HOG features...')

45.95 Seconds to extract HOG features...


#### Set up to train the classifier

In [4]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

# Randomly split up data into training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=rand_state)
    
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X_train)
# Apply the scaler to X
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))

Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 1764


#### Use Grid Search to automatically fit the classifer with the best parameters

The best parameters found through Grid Search was the RBF kernel with the penalty parameter (C) set to 35 and the kernel coefficient (gamma) set to automatic (1/n_features)

In [26]:
from sklearn.model_selection import GridSearchCV

t=time.time()

# define grid search parameters
# parameters = [{'C': [10], 'kernel': ['linear']}]
classifier = svm.SVC(C=35, kernel='rbf')

# run

# classifier = GridSearchCV(svc, parameters, verbose=10)
classifier.fit(X_train, y_train)

print(round(time.time()-t, 2), 'seconds to train...')

# Check the score of the SVC
print('Test Accuracy of SVC = ', round(classifier.score(X_test, y_test), 4))
# print('The optimal parameters determined by Grid Search are: ', classifier.best_params_)

70.87 seconds to train...
Test Accuracy of SVC =  0.9609


#### Save the trained model
Now that the model is working well, ~96% accuracy, comment out the save function so I don't accidentally save over my results.

In [9]:
from sklearn.externals import joblib
# joblib.dump(classifier, 'svm_model.pkl') 

['svm_model.pkl']

#### Restore the trained model

In [20]:
classifier = joblib.load('svm_model.pkl')

#### See how well our model generalizes and where it has trouble by generating the confusion matrix

In [21]:
# Predict the Test set results
y_pred = classifier.predict(X_test)

# Make the Confusion Matrix
from sklearn.metrics import confusion_matrix
# cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print("True negatives: ", tn, "\nFalse positives: ", fp, "\nFalse negatives: ", fn, "\nTrue positives: ", tp)

True negatives:  1730 
False positives:  75 
False negatives:  64 
True positives:  1683


We can see from the result of the confusion matrix that the model is performing in a very balanced manner. The frequency of falsely identifying a car and falsely predicting the absence of a car, are about equal.

#### Detection and tracking pipeline
Now that we havve a trained classifier, it's time to build a pipeline to accept images from a video stream, detct cars, and track them in subsequent frames. First we will define the functions needed to do these separate steps:

In [ ]:
def extract_from_img():
    pass

Then put the whole pipeline together to process a video frame

In [ ]:
def process_image(image):
    pass

Then finally add the functionality to grab single frames from a video:

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

output = 'test_videos_output/test.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip

clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
clip = clip1.fl_image(process_image) # NOTE: this function expects color images!!
%time clip.write_videofile(output, audio=False)